# Automated CapsnetsS2I train + test

First, import stuff

In [7]:
import model_s2i
import data_loader
import tensorflow as tf

In [8]:
import warnings
warnings.filterwarnings('ignore')
tf.logging.set_verbosity(tf.logging.ERROR)

## Load word vectors and data

In [9]:
WORD_VEC_SCENARIO = "FastText-300-verbsDatasets-synAntVSP-dia"
SCENARIO_NAME = 'no-rerouting'

word2vec_path = '../../romanian_word_vecs/cleaned-vectors.vec'

# No diacritics data
training_data_paths = [
#         '../data-capsnets/scenario0/train.txt',
#         '../data-capsnets/scenario1/train.txt',
#         '../data-capsnets/scenario2/train.txt',
#         '../data-capsnets/scenario3.1/train.txt',
#         '../data-capsnets/scenario3.2/train.txt',
#         '../data-capsnets/scenario3.3/train.txt',
        '../data-capsnets/diacritics/scenario0/train.txt',
        '../data-capsnets/diacritics/scenario1/train.txt',
        '../data-capsnets/diacritics/scenario2/train.txt',
        '../data-capsnets/diacritics/scenario31/train.txt',
        '../data-capsnets/diacritics/scenario32/train.txt',
        '../data-capsnets/diacritics/scenario33/train.txt',
        
    ]
test_data_paths = [
#     '../data-capsnets/scenario0/test.txt',
#     '../data-capsnets/scenario1/test.txt',
#     '../data-capsnets/scenario2/test.txt',
#     '../data-capsnets/scenario3.1/test.txt',
#     '../data-capsnets/scenario3.2/test.txt',
#     '../data-capsnets/scenario3.3/test.txt',
    '../data-capsnets/diacritics/scenario0/test.txt',
    '../data-capsnets/diacritics/scenario1/test.txt',
    '../data-capsnets/diacritics/scenario2/test.txt',
    '../data-capsnets/diacritics/scenario31/test.txt',
    '../data-capsnets/diacritics/scenario32/test.txt',
    '../data-capsnets/diacritics/scenario33/test.txt',
]

scenario_nums = [
#     '0_' + WORD_VEC_SCENARIO,
#     '1_' + WORD_VEC_SCENARIO,
#     '2_' + WORD_VEC_SCENARIO,
#     '31_' + WORD_VEC_SCENARIO,
#     '32_' + WORD_VEC_SCENARIO,
#     '33_' + WORD_VEC_SCENARIO,
#     '0_dia_' + WORD_VEC_SCENARIO,
#     '1_dia_' + WORD_VEC_SCENARIO,
#     '2_dia_' + WORD_VEC_SCENARIO,
#     '31_dia_' + WORD_VEC_SCENARIO,
#     '32_dia_' + WORD_VEC_SCENARIO,
#     '33_dia_' + WORD_VEC_SCENARIO,
    '0-no-rerouting',
    '1-no-rerouting',
    '2-no-rerouting',
    '31-no-rerouting',
    '32-no-rerouting',
    '33-no-rerouting',
]

In [10]:
print('------------------load word2vec begin-------------------')
w2v = data_loader.load_w2v(word2vec_path)
print('------------------load word2vec end---------------------')

------------------load word2vec begin-------------------
loading time took 000.06
------------------load word2vec end---------------------


# Flag setting functions + utils

In [11]:
import csv

def set_flags(data, scenario):
    FLAGS = tf.app.flags.FLAGS

    tf.app.flags.DEFINE_boolean('save_model', False, 'save model to disk')
    tf.app.flags.DEFINE_string('summaries_dir', './logs', 'tensorboard summaries')
    tf.app.flags.DEFINE_string('ckpt_dir', './saved_models/', 'check point dir')
    tf.app.flags.DEFINE_string('scenario_num', scenario, 'Scenario number')
    tf.app.flags.DEFINE_string('errors_dir', './errors/', 'Errors dir')
    tf.app.flags.DEFINE_string('results_dir', './results/', 'Results dir')
    tf.app.flags.DEFINE_string('hyperparams_dir', './hyperparams/', 'Hyperparams dir')
    
    vocab_size, word_emb_size = data['embedding'].shape
    _, max_sentence_length = data['x_tr'].shape
    intents_number = len(data['intents_dict'])
    slots_number = len(data['slots_dict'])
    hidden_size = 64

    tf.app.flags.DEFINE_float('keep_prob', 1, 'embedding dropout keep rate for training')
    tf.app.flags.DEFINE_integer('hidden_size', hidden_size, 'embedding vector size')
    tf.app.flags.DEFINE_integer('batch_size', 32, 'batch size')
    tf.app.flags.DEFINE_integer('num_epochs', 20, 'num of epochs')
    tf.app.flags.DEFINE_integer('vocab_size', vocab_size, 'vocab size of word vectors')
    tf.app.flags.DEFINE_integer('max_sentence_length', max_sentence_length, 'max number of words in one sentence')
    tf.app.flags.DEFINE_integer('intents_nr', intents_number, 'intents_number')  #
    tf.app.flags.DEFINE_integer('slots_nr', slots_number, 'slots_number')  #
    tf.app.flags.DEFINE_integer('word_emb_size', word_emb_size, 'embedding size of word vectors')
    tf.app.flags.DEFINE_boolean('use_embedding', True, 'whether to use embedding or not.')
    tf.app.flags.DEFINE_float('learning_rate', 0.01, 'learning rate')
    tf.app.flags.DEFINE_integer('slot_routing_num', 2, 'slot routing num')
    tf.app.flags.DEFINE_integer('intent_routing_num', 3, 'intent routing num')
    tf.app.flags.DEFINE_integer('intent_output_dim', 16, 'intent output dimension')
    tf.app.flags.DEFINE_integer('slot_output_dim', 2 * hidden_size, 'slot output dimension')
    tf.app.flags.DEFINE_integer('d_a', 20, 'self attention weight hidden units number')
    tf.app.flags.DEFINE_integer('r', 5, 'number of self attention heads')
    tf.app.flags.DEFINE_float('alpha', 0.0001, 'coefficient for self attention loss')
    tf.app.flags.DEFINE_integer('n_splits', 3, 'Number of cross-validation splits')
    tf.app.flags.DEFINE_float('rerouting_coef', 0.5, 'coefficient for rerouting')
    tf.app.flags.DEFINE_boolean('use_rerouting', False, 'whether to use rerouting or not')
    
    tf.app.flags.DEFINE_string('f', '', 'kernel') # Without this there's an error: unknown command line flag 'f'
    
    return FLAGS

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()
    keys_list = [keys for keys in flags_dict]
    for keys in keys_list:
        FLAGS.__delattr__(keys)
        
def write_csv(run_results):
    with open('results/results_{}.csv'.format(SCENARIO_NAME), 'w') as f:
        header_line = ['Scenario', 'Intent F1', 'Slot F1']
        writer = csv.writer(f)
        writer.writerow(header_line)
        for k, v in run_results.items():
            scenario_num = [k]
            intent_score = [v['intent_f1']]
            slot_score = [v['slot_f1']]
            l = scenario_num + intent_score + slot_score
            writer.writerow(l)

In [ ]:
# In case it's needed
del_all_flags(FLAGS)

In [12]:
import train
import test

results = dict()
for i in range(len(training_data_paths)):
    
    # Load data
    data = data_loader.read_datasets(w2v, training_data_paths[i], test_data_paths[i], test=True)
    
    FLAGS = set_flags(data, scenario=scenario_nums[i])
    
    # Train model
    print('TRAINING ' + scenario_nums[i])
    train.train(model_s2i.CapsNetS2I, data, FLAGS, batches_rand=False)
    
    # Test model
    print('TESTING ' + scenario_nums[i])
    intent_f, slot_f = test.test(model_s2i.CapsNetS2I, data, FLAGS)
    results[scenario_nums[i]] = dict()
    results[scenario_nums[i]]['intent_f1'] = intent_f
    results[scenario_nums[i]]['slot_f1'] = slot_f
        
    # Reset flags
    del_all_flags(FLAGS)

write_csv(results)

------------------read datasets begin-------------------
max length is 15
max length is 15
------------------read datasets end---------------------
TRAINING 0-no-rerouting
------------------start cross-validation-------------------
FOLD 1
Initializing Variables
using pre-trained word emebedding.begin...
using pre-trained word emebedding.ended...
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.076190
F score 0.076190
Slot filling
F1 score: 0.016983
Accuracy: 0.056508
------------------epoch :  0  Loss:  13.558519 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.573810
F score 0.573810
Slot filling
F1 score: 0.464832
Accuracy: 0.930952
Current F score mean 0.5193206640454346
Best F score mean 0.5193206640454346
------------------epoch :  1  Loss:  5.4807625 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.769048
F score 0.769048
Slot filling
F1 sc

------------------epoch :  5  Loss:  0.17416427 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.973810
F score 0.973810
Slot filling
F1 score: 0.992893
Accuracy: 0.999206
Current F score mean 0.983351462412376
Best F score mean 0.9955656595286986
------------------epoch :  6  Loss:  0.13919492 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.971429
F score 0.971429
Slot filling
F1 score: 0.994924
Accuracy: 0.999683
Current F score mean 0.9831762146482959
Best F score mean 0.9955656595286986
------------------epoch :  7  Loss:  0.11353427 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.990476
F score 0.990476
Slot filling
F1 score: 0.994924
Accuracy: 0.999683
Current F score mean 0.9927000241721055
Best F score mean 0.9955656595286986
------------------epoch :  8  Loss:  0.097661056 ----------------------
           VALIDA

           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.992857
F score 0.992857
Slot filling
F1 score: 0.984864
Accuracy: 0.999048
Current F score mean 0.9888604584114171
Best F score mean 0.9977932636469222
------------------epoch :  12  Loss:  0.06574765 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.992857
F score 0.992857
Slot filling
F1 score: 0.989899
Accuracy: 0.999365
Current F score mean 0.9913780663780665
Best F score mean 0.9977932636469222
------------------epoch :  13  Loss:  0.060525097 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.992857
F score 0.992857
Slot filling
F1 score: 0.989899
Accuracy: 0.999365
Current F score mean 0.9913780663780665
Best F score mean 0.9977932636469222
------------------epoch :  14  Loss:  0.056123 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.99285

<Figure size 640x480 with 2 Axes>

<Figure size 640x480 with 2 Axes>

...

           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.994949
F score 0.994949
Slot filling
F1 score: 0.977307
Accuracy: 0.996392
Current F score mean 0.9861283026941121
Best F score mean 0.9931714719271624
------------------epoch :  12  Loss:  0.20735538 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.994949
F score 0.994949
Slot filling
F1 score: 0.978788
Accuracy: 0.996753
Current F score mean 0.9868686868686869
Best F score mean 0.9931714719271624
------------------epoch :  13  Loss:  0.18605053 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.994949
F score 0.994949
Slot filling
F1 score: 0.978788
Accuracy: 0.996753
Current F score mean 0.9868686868686869
Best F score mean 0.9931714719271624
------------------epoch :  14  Loss:  0.16928545 ----------------------
           VALIDATION SET PERFORMANCE        
Intent detection
Intent accuracy 0.9949

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 2 Axes>

<Figure size 640x480 with 2 Axes>

Intent F1: 0.279070
Slot F1: 0.472248
